In [1]:
import numpy as np
import pandas as pd
import pickle
from itertools import product
pd.set_option('display.max_info_columns', 500)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_row', 300)
pd.set_option('display.float_format', lambda x: ' %.5f' % x)

In [2]:
train_dir = "./data/1_预处理/train/"
test_dir = "./data/1_预处理/A/"

特征_train_dir = "./data/2_特征/train/"
特征_test_dir = "./data/2_特征/A/"

In [3]:
df_train = {}
df_test = {}

In [4]:
# 创建 df_train 和 df_test
train_date_range = pd.date_range(
    start='2014-01-01 00:00:00',
    end='2019-12-31 23:00:00',  # 包含最后一日的23:00
    freq='h'  # 每小时频率
)
test_date_range = pd.date_range(
    start='2020-01-01 00:00:00',
    end='2021-12-31 23:00:00',  # 包含最后一日的23:00
    freq='h'  # 每小时频率
)

In [5]:
with open(train_dir + '雨量水位' + ".pkl" , "rb" ) as file :
    df_train['雨量水位'] = pickle.load(file)
with open(test_dir + '雨量水位' + ".pkl" , "rb" ) as file :
    df_test['雨量水位'] = pickle.load(file)

### 1、按照时间获取流量最大值、求和等数据

In [6]:
df_train_特征 = df_train['雨量水位'].groupby("时间")['值'].agg(['max' , 'min' , 'sum' , 'std']).reset_index()
df_test_特征 = df_test['雨量水位'].groupby("时间")['值'].agg(['max' , 'min' , 'sum' , 'std']).reset_index()

In [7]:
with open(特征_train_dir + '基础特征' + '.pkl' , "wb") as file:
    pickle.dump(df_train_特征, file)
with open(特征_test_dir + '基础特征' + '.pkl' , "wb") as file:
    pickle.dump(df_test_特征, file)

### 2、对于共有的所有检测站点，都求0，1，3，6，12，24，36，48小时内的流量情况。

In [22]:
共有监测站 = [i for i in df_train['雨量水位']['名称'].unique() if i in df_test['雨量水位']['名称'].unique()] 

In [23]:
# 定义所有 (统计量, 窗口) 组合
stats = ['max', 'min' , 'std' , 'sum' , 'mean' , 'skew']
windows = ['3h' , '6h' , '12h' , '24h' , '36h' , '48h']

In [24]:
df_特征_train = pd.DataFrame(train_date_range, columns=['时间'])
df = df_train['雨量水位'].copy()
df = df[df['名称'].isin(共有监测站)]
for 站点名称, group in df.groupby('名称', group_keys=False) :
    print(f"当前处理站点为: {站点名称}" , end='  ')  # 输出站点ID

    # 按时间排序并设置索引
    group = group.sort_values('时间').set_index('时间')
    
    # 用 product 生成所有组合
    for stat, window in product(stats, windows):
        group[f'{站点名称}_{stat}_{window}'] = group['值'].rolling(window, closed='both').agg(stat)
    group = group.drop(columns=['名称', '站点id', '值', '平均值', '最大值', '最大值发生时间', '最小值', '最小值发生时间']).reset_index()
    group = group.drop_duplicates(subset='时间', keep='first')
    # print(group)
    df_特征_train = df_特征_train.merge(group , on='时间' , how='left')

当前处理站点为: 万安镇雨量  当前处理站点为: 下保村雨量  当前处理站点为: 下墩村雨量  当前处理站点为: 下洋坂雨量  当前处理站点为: 下洋村雨量  当前处理站点为: 下路岭雨量  当前处理站点为: 东上村雨量  当前处理站点为: 东游镇雨量  当前处理站点为: 九丘雨量  当前处理站点为: 京口村雨量  当前处理站点为: 半山村雨量  当前处理站点为: 厚德村雨量  当前处理站点为: 双鲤村雨量  当前处理站点为: 台溪乡雨量  当前处理站点为: 后村雨量  当前处理站点为: 后洋雨量  当前处理站点为: 均中村雨量  当前处理站点为: 埔上镇雨量  当前处理站点为: 夏茂镇雨量  当前处理站点为: 夏阳乡雨量  当前处理站点为: 外洋村雨量  当前处理站点为: 大儒村雨量  当前处理站点为: 大华镇雨量  当前处理站点为: 大口窑雨量  当前处理站点为: 大横镇曲楼甲新农村雨量  当前处理站点为: 天心岛花园雨量  当前处理站点为: 安砂镇雨量  当前处理站点为: 官昌村雨量  当前处理站点为: 富屯溪流域面雨量  当前处理站点为: 小华村雨量  当前处理站点为: 小西门头社区雨量  当前处理站点为: 山下乡雨量  当前处理站点为: 山坊雨量  当前处理站点为: 岭坪雨量  当前处理站点为: 年坂山雨量  当前处理站点为: 广平镇雨量  当前处理站点为: 库区流域面雨量  当前处理站点为: 建溪流域面雨量  当前处理站点为: 文峰村雨量  当前处理站点为: 新桥头雨量  当前处理站点为: 新源村雨量  当前处理站点为: 新阳镇雨量  当前处理站点为: 景山新村雨量  当前处理站点为: 杉坜雨量  当前处理站点为: 李坊村雨量  当前处理站点为: 林后岗雨量  当前处理站点为: 柯坑村雨量  当前处理站点为: 桐木村雨量  当前处理站点为: 桥下村雨量  当前处理站点为: 桥南村雨量  当前处理站点为: 梅仙镇雨量  当前处理站点为: 梅园小区雨量  当前处理站点为: 梅山镇雨量  当前处理站点为: 梅树湾新村雨量  当前处理站点为: 梅溪村雨量  当前处理站点为: 梓溪村雨量  当前处理站点为: 槐南镇雨量  当前处理站点为: 樟树村雨量  当前处理站点为: 横坑村雨量  当前处理站点为: 横洋雨量  当前处理站点为: 民主村雨量  当前处理站点为: 

In [25]:
df_特征_test = pd.DataFrame(test_date_range, columns=['时间'])
df = df_test['雨量水位'].copy()
df = df[df['名称'].isin(共有监测站)]
for 站点名称, group in df.groupby('名称', group_keys=False) :
    print(f"当前处理站点为: {站点名称}" , end='  ')  # 输出站点ID

    # 按时间排序并设置索引
    group = group.sort_values('时间').set_index('时间')

    # 用 product 生成所有组合
    for stat, window in product(stats, windows):
        group[f'{站点名称}_{stat}_{window}'] = group['值'].rolling(window, closed='both').agg(stat)
    group = group.drop(columns=['名称', '站点id', '值', '平均值', '最大值', '最大值发生时间', '最小值', '最小值发生时间']).reset_index()
    group = group.drop_duplicates(subset='时间', keep='first')
    # print(group)
    df_特征_test = df_特征_test.merge(group , on='时间' , how='left')

当前处理站点为: 万安镇雨量  当前处理站点为: 下保村雨量  当前处理站点为: 下墩村雨量  当前处理站点为: 下洋坂雨量  当前处理站点为: 下洋村雨量  当前处理站点为: 下路岭雨量  当前处理站点为: 东上村雨量  当前处理站点为: 东游镇雨量  当前处理站点为: 九丘雨量  当前处理站点为: 京口村雨量  当前处理站点为: 半山村雨量  当前处理站点为: 厚德村雨量  当前处理站点为: 双鲤村雨量  当前处理站点为: 台溪乡雨量  当前处理站点为: 后村雨量  当前处理站点为: 后洋雨量  当前处理站点为: 均中村雨量  当前处理站点为: 埔上镇雨量  当前处理站点为: 夏茂镇雨量  当前处理站点为: 夏阳乡雨量  当前处理站点为: 外洋村雨量  当前处理站点为: 大儒村雨量  当前处理站点为: 大华镇雨量  当前处理站点为: 大口窑雨量  当前处理站点为: 大横镇曲楼甲新农村雨量  当前处理站点为: 天心岛花园雨量  当前处理站点为: 安砂镇雨量  当前处理站点为: 官昌村雨量  当前处理站点为: 富屯溪流域面雨量  当前处理站点为: 小华村雨量  当前处理站点为: 小西门头社区雨量  当前处理站点为: 山下乡雨量  当前处理站点为: 山坊雨量  当前处理站点为: 岭坪雨量  当前处理站点为: 年坂山雨量  当前处理站点为: 广平镇雨量  当前处理站点为: 库区流域面雨量  当前处理站点为: 建溪流域面雨量  当前处理站点为: 文峰村雨量  当前处理站点为: 新桥头雨量  当前处理站点为: 新源村雨量  当前处理站点为: 新阳镇雨量  当前处理站点为: 景山新村雨量  当前处理站点为: 杉坜雨量  当前处理站点为: 李坊村雨量  当前处理站点为: 林后岗雨量  当前处理站点为: 柯坑村雨量  当前处理站点为: 桐木村雨量  当前处理站点为: 桥下村雨量  当前处理站点为: 桥南村雨量  当前处理站点为: 梅仙镇雨量  当前处理站点为: 梅园小区雨量  当前处理站点为: 梅山镇雨量  当前处理站点为: 梅树湾新村雨量  当前处理站点为: 梅溪村雨量  当前处理站点为: 梓溪村雨量  当前处理站点为: 槐南镇雨量  当前处理站点为: 樟树村雨量  当前处理站点为: 横坑村雨量  当前处理站点为: 横洋雨量  当前处理站点为: 民主村雨量  当前处理站点为: 

In [26]:
df_特征_train = df_特征_train.fillna(0)
df_特征_test = df_特征_test.fillna(0)

In [12]:
df_特征_train.shape

(52584, 2497)

In [13]:
df_特征_test.shape

(17544, 2497)

In [14]:
with open(特征_train_dir + '1~36小时滑窗特征' + '.pkl' , "wb") as file:
    pickle.dump(df_特征_train, file)
with open(特征_test_dir + '1~36小时滑窗特征' + '.pkl' , "wb") as file:
    pickle.dump(df_特征_test, file)

### 获取年月日季节性的特征。

In [15]:
df_特征_train = pd.DataFrame(train_date_range, columns=['时间'])

df_特征_train['年份'] = df_特征_train['时间'].dt.year
df_特征_train['月份'] = df_特征_train['时间'].dt.month
df_特征_train['日'] = df_特征_train['时间'].dt.day
df_特征_train['小时'] = df_特征_train['时间'].dt.hour
df_特征_train['星期几'] = df_特征_train['时间'].dt.dayofweek  # 周一=0，周日=6
df_特征_train['季度'] = df_特征_train['时间'].dt.quarter
df_特征_train['是否周末'] = df_特征_train['星期几'].isin([5,6]).astype(int)

# 自定义水文时期特征
df_特征_train['水文时期'] = '正常期'
df_特征_train.loc[df_特征_train['月份'].isin([5,6,7]), '水文时期'] = '汛期'  # 汛期
df_特征_train.loc[df_特征_train['月份'].isin([10,11,12]), '水文时期'] = '枯水期'  # 枯水期

# 定义映射字典
season_mapping = {'正常期': 0, '汛期': 1, '枯水期': 2}

# 应用映射
df_特征_train['水文时期编码'] = df_特征_train['水文时期'].map(season_mapping)

# 或使用sklearn的LabelEncoder
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_特征_train['水文时期编码'] = le.fit_transform(df_特征_train['水文时期'])

df_特征_train = df_特征_train.drop(['水文时期'] , axis=1)

In [16]:
df_特征_test = pd.DataFrame(test_date_range, columns=['时间'])

df_特征_test['年份'] = df_特征_test['时间'].dt.year
df_特征_test['月份'] = df_特征_test['时间'].dt.month
df_特征_test['日'] = df_特征_test['时间'].dt.day
df_特征_test['小时'] = df_特征_test['时间'].dt.hour
df_特征_test['星期几'] = df_特征_test['时间'].dt.dayofweek  # 周一=0，周日=6
df_特征_test['季度'] = df_特征_test['时间'].dt.quarter
df_特征_test['是否周末'] = df_特征_test['星期几'].isin([5,6]).astype(int)

# 自定义水文时期特征
df_特征_test['水文时期'] = '正常期'
df_特征_test.loc[df_特征_test['月份'].isin([5,6,7]), '水文时期'] = '汛期'  # 汛期
df_特征_test.loc[df_特征_test['月份'].isin([10,11,12]), '水文时期'] = '枯水期'  # 枯水期

# 定义映射字典
season_mapping = {'正常期': 0, '汛期': 1, '枯水期': 2}

# 应用映射
df_特征_test['水文时期编码'] = df_特征_test['水文时期'].map(season_mapping)

# 或使用sklearn的LabelEncoder
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_特征_test['水文时期编码'] = le.fit_transform(df_特征_test['水文时期'])

df_特征_test = df_特征_test.drop(['水文时期'] , axis=1)

In [17]:
with open(特征_train_dir + '年月日季节' + '.pkl' , "wb") as file:
    pickle.dump(df_特征_train, file)
with open(特征_test_dir + '年月日季节' + '.pkl' , "wb") as file:
    pickle.dump(df_特征_test, file)

### 广东哥特征

In [18]:
df_特征_train = pd.DataFrame(train_date_range, columns=['时间'])
df = df_train['雨量水位'].copy()
# 按时间分组统计
time_group = df.groupby('时间')['值'].agg([
    ('v_sum', 'sum'),
    ('v_mean', 'mean'),
    ('v_std', 'std'),
    ('v_max', 'max'),
    ('v_min', 'min'),
    ('v_skew', 'skew')
]).reset_index()

# 生成差分特征
time_group['v_sum_diff'] = time_group['v_sum'].diff()
time_group['v_mean_diff'] = time_group['v_mean'].diff()

# 生成滚动窗口特征（示例：8~720小时窗口，步长8）
time_group = time_group.sort_values('时间')  # 确保时间有序
for w in range(8, 720+8, 8):
    time_group[f'v_mean_roll_{w}'] = time_group['v_mean'].rolling(
        window=w, 
        min_periods=1
    ).mean()
df_特征_train = df_特征_train.merge(time_group , on='时间' , how='left')

In [19]:
df_特征_test = pd.DataFrame(test_date_range, columns=['时间'])
df = df_test['雨量水位'].copy()
# 按时间分组统计
time_group = df.groupby('时间')['值'].agg([
    ('v_sum', 'sum'),
    ('v_mean', 'mean'),
    ('v_std', 'std'),
    ('v_max', 'max'),
    ('v_min', 'min'),
    ('v_skew', 'skew')
]).reset_index()

# 生成差分特征
time_group['v_sum_diff'] = time_group['v_sum'].diff()
time_group['v_mean_diff'] = time_group['v_mean'].diff()

# 生成滚动窗口特征（示例：8~720小时窗口，步长8）
time_group = time_group.sort_values('时间')  # 确保时间有序
for w in range(8, 720+8, 8):
    time_group[f'v_mean_roll_{w}'] = time_group['v_mean'].rolling(
        window=w, 
        min_periods=1
    ).mean()
df_特征_test = df_特征_test.merge(time_group , on='时间' , how='left')

In [20]:
with open(特征_train_dir + '广东哥特征' + '.pkl' , "wb") as file:
    pickle.dump(df_特征_train, file)
with open(特征_test_dir + '广东哥特征' + '.pkl' , "wb") as file:
    pickle.dump(df_特征_test, file)

In [21]:
for i in df_特征_train.columns :
    print(i)

时间
v_sum
v_mean
v_std
v_max
v_min
v_skew
v_sum_diff
v_mean_diff
v_mean_roll_8
v_mean_roll_16
v_mean_roll_24
v_mean_roll_32
v_mean_roll_40
v_mean_roll_48
v_mean_roll_56
v_mean_roll_64
v_mean_roll_72
v_mean_roll_80
v_mean_roll_88
v_mean_roll_96
v_mean_roll_104
v_mean_roll_112
v_mean_roll_120
v_mean_roll_128
v_mean_roll_136
v_mean_roll_144
v_mean_roll_152
v_mean_roll_160
v_mean_roll_168
v_mean_roll_176
v_mean_roll_184
v_mean_roll_192
v_mean_roll_200
v_mean_roll_208
v_mean_roll_216
v_mean_roll_224
v_mean_roll_232
v_mean_roll_240
v_mean_roll_248
v_mean_roll_256
v_mean_roll_264
v_mean_roll_272
v_mean_roll_280
v_mean_roll_288
v_mean_roll_296
v_mean_roll_304
v_mean_roll_312
v_mean_roll_320
v_mean_roll_328
v_mean_roll_336
v_mean_roll_344
v_mean_roll_352
v_mean_roll_360
v_mean_roll_368
v_mean_roll_376
v_mean_roll_384
v_mean_roll_392
v_mean_roll_400
v_mean_roll_408
v_mean_roll_416
v_mean_roll_424
v_mean_roll_432
v_mean_roll_440
v_mean_roll_448
v_mean_roll_456
v_mean_roll_464
v_mean_roll_472
v_mea